# Chap.09 教師信号あり総合演習

## 9.2 練習問題：金融機関のキャンペーン分析

データ分析の方針検討

In [ ]:
# EX.9-1
# 教師あり学習の分類

In [ ]:
# EX.9-2
### 解答例
# ｙ列を予測する分類モデルを作ることにより、その顧客が購入してくれるかどうかが事前に分かる。
# また、特徴量にこちらからのアプローチ法を入れる事により、「この顧客の場合、こういうアプローチをとったら購入してくれる」というような営業の最適化を行う事が出来る


分析の実施

In [1]:
# ライブラリの準備
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
# EX.9-3 データを取得する
df = pd.read_csv('../support/datafiles/Bank.csv')
print(df.head, "\n")
print(df.tail)
df.shape